# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

0


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'MobileNetV2'
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

In [6]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
train_data_dir = "/hd/jbpark/dataset/ms1m/"
tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, binary_img=binary_img, dataset_len=dataset_len,
    is_ccrop=is_ccrop)

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.summary()

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Functiona (None, 4, 4, 1280)   2257984     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    mobilenetv2_1.00_224[0][0]       
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [5]:
from pathlib import Path

base_dir = "/hd/jbpark/models/"
save_name = "ms1m_check/"
if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
callbacks = [mc_callback, tb_callback]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 1:12:37 - loss: 46.4035
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_46.271385192871094.ckpt
 2000/36391 [>.............................] - ETA: 1:10:46 - loss: 46.2286
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_45.82712173461914.ckpt
 3000/36391 [=>............................] - ETA: 1:08:50 - loss: 46.0116
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_45.322776794433594.ckpt
 4000/36391 [==>...........................] - ETA: 1:06:53 - loss: 45.7751
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_44.806636810302734.ckpt
 5000/36391 [===>..........................] - ETA: 1:04:51 - loss: 45.5303
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_44.296119689941406.ckpt
 6000/36391 [===>........

### Resume training with latest checkpoint

In [ ]:
from glob import glob
base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
best_checkpoint

In [7]:
from pathlib import Path

base_dir = "/hd/jbpark/models/"
save_name = "ms1m_check/"
if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    ### MS1M dataset
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'

    train_dataset = dataset.load_tfrecord_dataset(
        tfrecord_name, batch_size, binary_img=binary_img,
        is_ccrop=is_ccrop)
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/resume_e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=0.1, min_lr=0.001)
callbacks = [mc_callback, tb_callback,reduce_lr]

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [8]:
model.fit(
    train_dataset, 
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks
)

Epoch 1/20
 1000/45489 [..............................] - ETA: 1:33:45 - loss: 43.5909
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/resume_e_1_l_43.36653137207031.ckpt
 2000/45489 [>.............................] - ETA: 1:32:18 - loss: 43.3574
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/resume_e_1_l_42.87136459350586.ckpt
 3000/45489 [>.............................] - ETA: 1:30:28 - loss: 43.1108
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/resume_e_1_l_42.358985900878906.ckpt
 4000/45489 [=>............................] - ETA: 1:28:30 - loss: 42.8592
Epoch 00001: saving model to /hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/resume_e_1_l_41.855953216552734.ckpt
 4213/45489 [=>............................] - ETA: 1:28:08 - loss: 42.8062

KeyboardInterrupt: 

# Training Face Identification Classifier

# TensorFlow Lite Conversion

In [14]:
import numpy as np
import tensorflow as tf
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from losses import softmax_loss
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_16.tflite', 'wb') as f:
    f.write(tflite_model)

KeyboardInterrupt: 